# Implementation of the CMFS algorithm for feature selection

### Toy implementation

In [11]:
from gensim.corpora import mmcorpus, Dictionary
import numpy as np
import operator

In [12]:
texts = [['human', 'interface', 'computer'],
         ['survey', 'user', 'computer', 'system', 'response', 'time'],
         ['eps', 'user', 'interface', 'system'],
         ['system', 'human', 'system', 'eps'],
         ['user', 'response', 'time'],
         ['trees'],
         ['graph', 'trees'],
         ['graph', 'minors', 'trees'],
         ['graph', 'minors', 'survey']]

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

### Rows are all the unique tokens, columns are all the documents

In [13]:
rows = len(dictionary.keys())
cols = dictionary.num_docs
print "rows : {0}\ncols : {1}".format(rows, cols)

rows : 12
cols : 9


### Initialize matrix

In [14]:
matrix = np.zeros((rows, cols))
print matrix

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]]


### Algorithm

For each term, CMFS is calculated for each document as:
CMFS(t_k, c_i) = [(tf(t_k, c_i) + 1) ^ 2] / [(tf(t_k) + num_docs) * (tf(t, c_i) + num_terms)]

### Perform first pass. Create term-to-category feature-appearance matrix

In [15]:
# Step 1: Obtain sum of all features in category c_i
# Step 2: Obtain frequency of feature t_k in the entire training set
# Step 3: Obtain frequency of t_k in category c_i
# First two steps can be performed later by acting on the matrix.
for docno, doc in enumerate(corpus):
    for term, freq in doc:
        matrix[term][docno] = freq

print matrix

[[ 1.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  1.  1.  2.  0.  0.  0.  0.  0.]
 [ 0.  1.  1.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  1.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  1.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  1.]]


### Calculate CMFS for each term for each category

In [16]:
# Calculate total number of terms in each doc
term_freq_per_doc = np.cumsum(matrix, axis=0)[-1, :]
for term in range(rows):
    # Frequency of the term across all docs
    total_term_freq = sum(matrix[term, :])
    for doc in range(cols):
        numerator = float((matrix[term][doc] + 1) ** 2)
        denominator = (total_term_freq + cols) * (term_freq_per_doc[doc] + rows)
        matrix[term][doc] = numerator / denominator
        
# Final CMFS matrix
print matrix

[[ 0.02424242  0.00505051  0.02272727  0.00568182  0.00606061  0.00699301
   0.00649351  0.00606061  0.00606061]
 [ 0.02424242  0.02020202  0.00568182  0.00568182  0.00606061  0.00699301
   0.00649351  0.00606061  0.00606061]
 [ 0.02424242  0.00505051  0.00568182  0.02272727  0.00606061  0.00699301
   0.00649351  0.00606061  0.00606061]
 [ 0.00606061  0.02020202  0.00568182  0.00568182  0.02424242  0.00699301
   0.00649351  0.00606061  0.00606061]
 [ 0.00606061  0.02020202  0.00568182  0.00568182  0.02424242  0.00699301
   0.00649351  0.00606061  0.00606061]
 [ 0.00606061  0.02020202  0.00568182  0.00568182  0.00606061  0.00699301
   0.00649351  0.00606061  0.02424242]
 [ 0.00512821  0.01709402  0.01923077  0.04326923  0.00512821  0.00591716
   0.00549451  0.00512821  0.00512821]
 [ 0.00555556  0.01851852  0.02083333  0.00520833  0.02222222  0.00641026
   0.00595238  0.00555556  0.00555556]
 [ 0.00606061  0.00505051  0.02272727  0.02272727  0.00606061  0.00699301
   0.00649351  0.00606

### Extract top 5 features

 ### Formula used here to get CMFS is: CMFS_max(t_k) = max CMFS of term over all docs

In [17]:
term_cmfs_dict = {}
cmfs_max = np.max(matrix, axis=1)
for i in range(rows):
    term = dictionary[i]
    term_cmfs_dict[term] = cmfs_max[i]

print term_cmfs_dict

{u'minors': 0.024242424242424242, u'graph': 0.023809523809523808, u'system': 0.043269230769230768, u'trees': 0.02564102564102564, u'eps': 0.022727272727272728, u'computer': 0.024242424242424242, u'survey': 0.024242424242424242, u'user': 0.022222222222222223, u'human': 0.024242424242424242, u'time': 0.024242424242424242, u'interface': 0.024242424242424242, u'response': 0.024242424242424242}


### To extract top 5 features, simply select the terms with top 5 CMFS scores

In [18]:
sorted_feature_list = sorted(term_cmfs_dict.items(), key=operator.itemgetter(1), reverse=True)[:5]
print "-------Selected features are-------\n"
for term, cmfs in sorted_feature_list:
    print "Term: {0} \t CMFS: {1}".format(term, cmfs)

-------Selected features are-------

Term: system 	 CMFS: 0.0432692307692
Term: trees 	 CMFS: 0.025641025641
Term: minors 	 CMFS: 0.0242424242424
Term: computer 	 CMFS: 0.0242424242424
Term: survey 	 CMFS: 0.0242424242424
